<a href="https://colab.research.google.com/github/krrish9669/machine-learning-new/blob/main/Titanic_PipeLine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#MissingValues->OneHot->Scaling->FeatureSelection->Train_Using_Decision_Tree
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("brendan45774/test-file")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'test-file' dataset.
Path to dataset files: /kaggle/input/test-file


In [3]:
import os
os.listdir(path)


['tested.csv']

In [4]:
df1 = pd.read_csv(path+"/tested.csv")
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
df1.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace= True)

In [7]:
# Train/test/split
X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(df1.drop(columns=['Survived']),df1['Survived'],test_size=0.2,random_state=42)

In [8]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


Step 1 Missing Values: imputation transformer (column transformer)

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
trf1= ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_fair',SimpleImputer(strategy='most_frequent'),[5])
],remainder='passthrough')

we use col indeex instead of their name cuz when the step 1 finishes it wil return an array not a df hence to get a working flow we will use index

Step 2 OneHot Encoding

In [13]:
from sklearn.preprocessing import OneHotEncoder
trf2= ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

Step 3 Scaling

In [15]:
from sklearn.preprocessing import MinMaxScaler
trf3= ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])
"slice 10 because sex become 2 and embarked becomes 3"

Step 4 Feature Scaling


In [16]:
from sklearn.feature_selection import SelectKBest, chi2
trf4= SelectKBest(score_func=chi2,k=8)

step 5 Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier
trf5 = DecisionTreeClassifier()

MAIN STEP TO PUT ALL STEPS TOGETHER USING " PIPELINE "

In [19]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])
"u can also use make_pipeline: pipe = make_pipeline(trf1,,,trfn)"

'u can also use make_pipeline: pipe = make_pipeline(trf1,,,trfn)'

NOW WE TRAIN THE PIPE OBJECT

In [20]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_fair',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x79a0f85e5b20>)),
                ('trf5', DecisionTreeClassifier())])

if ur pipe line has an algo in the end use : .fit
if it doesnt have a algo in end: .fir_transform

In [21]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_fair',
                                  SimpleImputer(strategy='most_frequent'),
                                  [5])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x79a0f85e5b20>),
 'trf5': DecisionTreeClassifier()}

In [22]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array([7.75])

In [23]:
y_pred = pipe.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
#less because feature selection is harming our prediction

0.5952380952380952

CrossValidation using pipeline

In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.6437358661239257)

GridSearch using Pipeline(hyperparameter tuning)

In [27]:
params = {'trf5__max_depth':[1,2,3,4,5,None]}

In [28]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring = 'accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_fair',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [5])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x79a0f85e5b20>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [30]:
grid.best_score_

np.float64(0.6467209407507915)

In [31]:
grid.best_params_

{'trf5__max_depth': 1}